In [2]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

# Create engine
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")

# Create DataFrame from Facilities table
facilities = pd.read_sql_query(
    "SELECT * \
    FROM Facilities",
    engine)

# Verify results
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [3]:
# # Create DataFrame from Members table
members = pd.read_sql_query(
    "SELECT * \
    FROM Members",
    engine
)

# Verify results
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [4]:
# Create DataFrame from Bookings table
bookings = pd.read_sql_query(
    "SELECT * \
    FROM Bookings",
    engine
)

# Verify results
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


#### **A10:**

In [6]:
# Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue.
# Remember that there's a different cost for guests and members!

SELECT = " \
f.name AS facility, \
CASE WHEN m.memid = 0 THEN (f.guestcost * b.slots) \
    ELSE (f.membercost * b.slots) END AS revenue"
FROM = "Bookings AS b"
LEFT_JOIN_1 = "Members AS m"
USING_1 = "(memid)"
LEFT_JOIN_2 = "Facilities AS f"
USING_2 = "(facid)"
GROUP_BY = "facility"
HAVING = "revenue < 1000"
ORDER_BY = "revenue"

query = f" \
SELECT {SELECT} \
FROM {FROM} \
LEFT JOIN {LEFT_JOIN_1} \
USING{USING_1} \
LEFT JOIN {LEFT_JOIN_2} \
USING{USING_2} \
GROUP BY {GROUP_BY} \
HAVING {HAVING} \
ORDER BY {ORDER_BY}"

df = pd.read_sql_query(query, engine)

df

,facility,revenue
0,Badminton Court,0.0
1,Pool Table,0.0
2,Snooker Table,0.0
3,Table Tennis,0.0
4,Tennis Court 1,15.0
5,Massage Room 1,19.8
6,Squash Court,35.0
7,Tennis Court 2,75.0
8,Massage Room 2,160.0


#### **A11:**

In [8]:
# Produce a report of members and who recommended them in alphabetic surname,firstname order

SELECT = " \
    sq.memid, \
    sq.firstname, \
    sq.surname, \
    sq.recommendedby, \
    m.firstname, \
    m.surname"
FROM = "( \
		SELECT \
    		memid, \
			surname, \
			firstname, \
			recommendedby \
		FROM Members as m \
    						) AS sq"
LEFT_JOIN = "Members AS m"
ON = "sq.recommendedby = m.memid"
WHERE = "sq.memid <> 0"
ORDER_BY = "sq.surname, sq.firstname, m.surname, m.firstname"

query = f" \
SELECT {SELECT} \
FROM {FROM} \
LEFT JOIN {LEFT_JOIN} \
ON {ON} \
WHERE {WHERE} \
ORDER BY {ORDER_BY}"

df = pd.read_sql_query(query, engine)

df.head()

,memid,firstname,surname,recommendedby,firstname,surname
0,15,Florence,Bader,9,Ponder,Stibbons
1,12,Anne,Baker,9,Ponder,Stibbons
2,16,Timothy,Baker,13,Jemima,Farrell
3,8,Tim,Boothe,3,Tim,Rownam
4,5,Gerald,Butters,1,Darren,Smith


#### **A12:**

In [10]:
# Find the facilities with their usage by member, but not guests

SELECT = " \
	m.firstname AS firstname, \
	m.surname AS surname, \
	f.name AS facility, \
	SUM(b.slots) AS uses"
FROM = "Bookings AS b"
LEFT_JOIN_1 = "Members AS m"
USING_1 = "(memid)"
LEFT_JOIN_2 = "Facilities AS f"
USING_2 = "(facid)"
WHERE = "m.memid <> 0"
GROUP_BY = "firstname, surname, facility"
ORDER_BY = "surname, firstname, facility"

query = f" \
SELECT {SELECT} \
FROM {FROM} \
LEFT JOIN {LEFT_JOIN_1} \
USING{USING_1} \
LEFT JOIN {LEFT_JOIN_2} \
USING{USING_2} \
WHERE {WHERE} \
GROUP BY {GROUP_BY} \
ORDER BY {ORDER_BY}"

df = pd.read_sql_query(query, engine)

df.head()

,firstname,surname,facility,uses
0,Florence,Bader,Badminton Court,27
1,Florence,Bader,Massage Room 2,4
2,Florence,Bader,Pool Table,23
3,Florence,Bader,Snooker Table,66
4,Florence,Bader,Squash Court,4


#### **A13:**

In [12]:
# Find the facilities usage by month, but not guests

SELECT = " \
	strftime('%m', b.starttime) AS month, \
	f.name AS facility, \
	SUM(b.slots) AS uses"
FROM = "Bookings AS b"
LEFT_JOIN_1 = "Members AS m"
USING_1 = "(memid)"
LEFT_JOIN_2 = "Facilities AS f"
USING_2 = "(facid)"
WHERE = "m.memid <> 0"
GROUP_BY = "month, facility"
ORDER_BY = "month, facility"

query = f" \
SELECT {SELECT} \
FROM {FROM} \
LEFT JOIN {LEFT_JOIN_1} \
USING{USING_1} \
LEFT JOIN {LEFT_JOIN_2} \
USING{USING_2} \
WHERE {WHERE} \
GROUP BY {GROUP_BY} \
ORDER BY {ORDER_BY}"

df = pd.read_sql_query(query, engine)

df.head()

,month,facility,uses
0,07,Badminton Court,165
1,07,Massage Room 1,166
2,07,Massage Room 2,8
3,07,Pool Table,110
4,07,Snooker Table,140
